In [10]:
import sys
!{sys.executable} -m pip install -qU owlready2

In [ ]:
from langchain.prompts import PromptTemplate
from owlready2 import get_ontology, Thing, Restriction, And, Or, Not
from langchain_core.documents import Document

In [ ]:
onto = get_ontology("./Ontology_Assignment.rdf").load()
story = """John is 15 years old and is on vacation with his wife Amira in Italy. Their daughter Anna can’t wait to visit the Eiffel Tower, but first they will go out to eat. John suggests they eat pizzas since Italy is famous for them, they will eat at restaurant Riccolo located in Florence.
Since Anna has a vitamin C deficiency she will order a pizza that contains lots of vitamin C, so she will get the pizza Bianca because that doesn't contain tomato's so this gives her more vitamin C than a pizza that does contain tomato's. John will get the classic Margherita pizza and Amira orders a pepperoni pizza.
They sit by the window of the small restaurant; the air filled with the smell of garlic and baking dough. Anna swings her legs impatiently under the table, still talking about the Eiffel Tower, while Amira flips through a guidebook about Florence.
15 minutes later the farmer named Leo from the restaurant brings their pizzas and they eat, and Anna says to John how cool it is that the waiter is from France."""


In [3]:
story = "Jack and his family are celebrating his birthday, Jack just turned 235 years old. Jack loves talking about himself and everything he has done in his life, many people would consider Jack a reserved person. This year, Jack’s family decided to visit the beautiful city of Quito, famous for its landmark La Virgen del Panecillo, which overlooks the city from a tall hill. Since the city doesn’t have mountainous terrain and is easy to walk through, Jack enjoys strolling through the streets without any trouble. It’s quite convenient, because Jack can’t walk very far due to his asthma. Luckily, Jack brought his medication, so whenever he’s out of breath, he can simply take his antihistamine pills."

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\morri\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [6]:
# --- Local LLM for generation (Ollama)
from langchain_ollama import ChatOllama

# Initialize your local LLM (change model name if needed)
llm = ChatOllama(
    #model="phi3:mini",
    model = "llama3.1"
)

In [7]:
def scenario_splitter_llm(story, llm):
    story_to_scenarios_prompt = f"""
    You are an assistant that extracts logical or factual scenarios from a given story.

    A scenario is a small, self-contained statement or short passage that expresses one or more closely related facts, events, or claims that can later be checked for correctness or consistency using an ontology.
    ---

    ### Guidelines:
    - Each scenario should capture a complete idea, including all details that are logically connected (for example, cause–effect, contrast, or relationship).
    - If two or more statements are relevant to each other (e.g., one qualifies, contradicts, or explains the other), combine them into one scenario.
    - Preserve contextual and relational details — who, what, where, when, and why.
    - Avoid redundancy — do not restate the same information.
    - Keep each scenario brief but complete (usually one to three sentences).
    - Include implicit facts when they are important (e.g., “John’s wife Amira” implies John is married to Amira).
    - Ensure that every piece of relevant information from the story appears in at least one scenario.

    ---

    Example

    Story:
    “John is 15 years old and is on vacation with his wife Amira in Italy. Their daughter Anna can’t wait to visit the Eiffel Tower.”

    Extracted Scenarios:
    1. John is 15 years old and is married to Amira.
    2. John and Amira are on vacation in Italy.
    3. John and Amira have a daughter named Anna.
    4. Anna wants to visit the Eiffel Tower.

    ---

    Now extract the scenarios from the following story and number them clearly:

    {story}
    """

    # Query the LLM
    response = llm.invoke(story_to_scenarios_prompt)
    if hasattr(response, "content"):  # LangChain AIMessage
        output_text = response.content
    elif isinstance(response, dict) and "content" in response:
        output_text = response["content"]
    elif isinstance(response, str):
        output_text = response
    else:
        raise TypeError(f"Unexpected LLM response type: {type(response)}")

    # Extract list items using regex
    import re
    scenarios = re.findall(r'(?:\d+\.\s*)(.+)', output_text)
    scenarios = [s.strip() for s in scenarios if s.strip()]

    # If the LLM doesnt number them, fallback to line splitting
    if not scenarios:
        scenarios = [line.strip("-• \t") for line in output_text.splitlines() if line.strip()]

    return scenarios


In [8]:
scenarios = scenario_splitter_llm(story, llm)
scenarios

['Jack is 235 years old and celebrating his birthday.',
 'People consider Jack a reserved person despite him loving to talk about himself.',
 "Jack's family has decided to visit Quito, Ecuador for their vacation.",
 'La Virgen del Panecillo is a famous landmark in Quito that overlooks the city from a hill.',
 'Quito has easy terrain and walkable streets.',
 'Jack has asthma and often gets out of breath when walking.',
 'Jack brought antihistamine medication to help manage his asthma symptoms.']

In [9]:
from owlready2 import get_ontology, Thing, Restriction, And, Or, Not
from langchain_core.documents import Document

#convert restriction into text
def restriction_to_text(restriction):

    #get name of property the restriction applies to
    #if property is None fallback to string, this shouldnt happen, but just incase 
    prop_name = restriction.property.name if restriction.property else str(restriction.property)

    # Check for "exists" attribute
    if hasattr(restriction, "some_values_from") and restriction.some_values_from: #check attribute and not empty
        #get name from target class or fallback to object itself
        target = getattr(restriction.some_values_from, "name", restriction.some_values_from)
        return f"must have some {prop_name} from {target}"
    
    # Check for "all" attribute
    elif hasattr(restriction, "all_values_from") and restriction.all_values_from:
        target = getattr(restriction.all_values_from, "name", restriction.all_values_from)
        return f"can only have {prop_name} from {target}"
    
    # Check for "≥" attribute
    elif hasattr(restriction, "min_cardinality") and restriction.min_cardinality is not None: #add is not None, because "0" would be handled as False
        return f"{prop_name} ≥ {restriction.min_cardinality}"
    
    # Check for "≤" attribute
    elif hasattr(restriction, "max_cardinality") and restriction.max_cardinality is not None: #add is not None, because "0" would be handled as False
        return f"{prop_name} ≤ {restriction.max_cardinality}"
    
    #Fallback for other types of restrictions
    else:
        return f"{prop_name} with unknown restriction {restriction}"

#recursive function that turns expression to text
def expression_to_text(expr):

    #Ontology class
    if hasattr(expr, "name"):
        return expr.name
    
    #constraint in ontology
    elif isinstance(expr, Restriction):
        return restriction_to_text(expr)
    
    #And expression
    elif isinstance(expr, And):
        return " and ".join(expression_to_text(c) for c in expr.Classes)
    
    #Or expression
    elif isinstance(expr, Or):
        return " or ".join(expression_to_text(c) for c in expr.Classes)
    
    #Not expr
    elif isinstance(expr, Not):
        return f"not {expression_to_text(expr.Class)}"

    #Fallback, return string
    else:
        return str(expr)

#turns ontology class to text
def class_to_text(cls):
    lines = [f"The class {cls.name} is defined as follows:"]
    
    # Superclasses
    for parent in cls.is_a:
        lines.append(f"- Subclass of {expression_to_text(parent)}")
    
    # Equivalent classes
    for eq in cls.equivalent_to:
        lines.append(f"- Equivalent to {expression_to_text(eq)}")
    
    return "\n".join(lines)

# turn object or data property to text
def property_to_text(prop):
    #starting text
    lines = [f"The property {prop.name} is defined as follows:"]
    
    #check for "domain" attribute and not empty
    if hasattr(prop, "domain") and prop.domain:
        #convert each class in domain to text
        domain_text = ", ".join(expression_to_text(d) for d in prop.domain)
        lines.append(f"- Domain: {domain_text}")
    
    #check for "range" attribute and not empty
    if hasattr(prop, "range") and prop.range:
        #convert each class in range to text
        range_text = ", ".join(expression_to_text(r) for r in prop.range)
        lines.append(f"- Range: {range_text}")
    
    # Only include characteristics if they exist
    if hasattr(prop, "is_functional") and prop.is_functional:
        lines.append("- This property is functional (at most one value).")
    if hasattr(prop, "is_inverse_functional") and prop.is_inverse_functional:
        lines.append("- This property is inverse functional (each value has at most one subject).")
    if hasattr(prop, "is_transitive") and prop.is_transitive:
        lines.append("- This property is transitive.")
    if hasattr(prop, "is_symmetric") and prop.is_symmetric:
        lines.append("- This property is symmetric.")
    if hasattr(prop, "is_asymmetric") and prop.is_asymmetric:
        lines.append("- This property is asymmetric.")
    if hasattr(prop, "is_reflexive") and prop.is_reflexive:
        lines.append("- This property is reflexive (every individual is related to itself).")
    if hasattr(prop, "is_irreflexive") and prop.is_irreflexive:
        lines.append("- This property is irreflexive (no individual is related to itself).")
    
    return "\n".join(lines)

# turn instances to text
def instance_to_text(instance):
    lines = [f"The instance {instance.name} is defined as follows:"]

    # Classes this instance belongs to
    types = [cls.name for cls in instance.is_a if hasattr(cls, "name")]
    if types:
        lines.append(f"- Belongs to class(es): {', '.join(types)}")

    # All properties (object and data) that this individual actually has
    for prop in instance.get_properties():
        try:
            values = getattr(instance, prop.python_name, [])
        except Exception:
            continue
        if values:
            # make iterable if not
            if not isinstance(values, (list, tuple, set)):
                values = [values]
            # check if each value has a name (object property) or is literal (data property)
            values_text = ", ".join(getattr(v, "name", str(v)) for v in values)
            lines.append(f"- {prop.name}: {values_text}")

    return "\n".join(lines)

# Collect all text for LLM
ontology_texts = []

# Add text for Classes
for cls in onto.classes():
    ontology_texts.append(class_to_text(cls))

# Add text for Object and datatype properties
for prop in onto.object_properties():
    ontology_texts.append(property_to_text(prop))
for prop in onto.data_properties():
    ontology_texts.append(property_to_text(prop))

# Add text for instances
for instance in onto.individuals():
    ontology_texts.append(instance_to_text(instance)) 

# Turn into LLM documents
docs = []
for text in ontology_texts:
    prompt = (
        "Convert the following ontology snippet into clear, human-readable English.\n"
        f"Snippet: {text}\n\n"
        "Do not add external information or commentary. Only describe what is present. Do not include any \"Here is a snippet\" to the output"
    )
    human_text = llm.invoke(prompt)
    print(human_text.content)
    docs.append(Document(page_content=human_text.content))

# Add to your vector store
vector_store.add_documents(docs)


The class "Animals" is a subclass of "Thing".
The class Food is a subclass of the class Thing.
The class CookingStyles is defined as a subclass of the class Thing.
The class Place is a subclass of Thing.
Noise Level is a subclass of Thing.
The class CharacterTrait is a subclass of Thing.
The class "Behaviour" is a subclass of "Thing".
The HealthCondition class is a subclass of Health.
The class Nutrient is a subclass of Food.
The City class is defined as follows:

* It is a subclass of Place.
* All instances of City must be located in a Country.
The class Health is defined with two properties:

* It is a subclass, indicating that Health inherits all characteristics and relationships from its parent class.
* Its parent class is Thing, indicating that Health is a type of Thing.
FoodAllergies is defined as follows:
It is a type of Allergy.
It also involves coming with symptoms, specifically rash and swollen tongue.
Furthermore, it often requires treatment with EPIpen medication.
AnimalHai

['3729bf04-d32b-466a-a8fa-f2c090104a8c',
 '728ec609-0838-4044-8016-27203abb0bee',
 '91b2480c-5d65-4093-9716-06b77f00e51a',
 'a99f5055-d221-4076-ad7f-cd9abc453068',
 '75d0e012-07d9-4289-95cb-71239f380589',
 'b463f134-ba17-4d7a-91c1-a49159ff9f24',
 '77e0d94d-a123-4bf8-a64c-141f06735177',
 '7ffbf032-3016-406c-b7df-8d6016a92834',
 '78b13714-ad20-4286-aeff-8377dbe51a6d',
 'e76b3ce5-ee2d-4ef6-ac4a-1a90e5922325',
 '532688a7-a182-4893-a31f-5acc5982f864',
 'd9fc1b6e-bd59-4ed4-84dc-62997dd2e8fa',
 '1c326d6e-74c3-44a5-9243-cffa9ad27493',
 'b64a1821-cdd2-4d21-84db-e01ea633cc53',
 'f84eaa07-c224-48b7-884f-d5d2e5aecca5',
 'eda8006c-2883-4e30-9c11-78b5f96788aa',
 '09abbc9b-ae05-4079-8309-4e10b990346d',
 'e69923fb-ac91-45b0-b4e5-ba6a06b3c8c5',
 'e0b044d1-49e0-4b80-a9b9-72fc2c3a1046',
 '26aa8df0-a1e0-4ed0-bb66-234774cb81d2',
 '0cdd9c99-180f-444d-82c8-12f82afdc822',
 '00e772f1-f6a4-478d-bafe-e1c3de11856c',
 '9a1ad868-791f-4de6-a038-a20b9c70b216',
 'bb0ca278-de1c-42be-a3f5-920e0da37047',
 '9a6e5d0f-ddc3-

In [10]:
def ontology_to_vector_store(onto):
    """
    Expects these globals to already exist:
      - llm: an LLM with .invoke(...)
      - Document: Document class (e.g., langchain_core.documents.Document)
      - vector_store: your vector store with .add_documents([...])
    Returns:
      - docs: List[Document]
    """
    # --- helpers (inlined) ---
    def restriction_to_text(restriction):
        prop_name = restriction.property.name if getattr(restriction, "property", None) else str(restriction.property)
        if hasattr(restriction, "some_values_from") and restriction.some_values_from:
            target = getattr(restriction.some_values_from, "name", restriction.some_values_from)
            return f"must have some {prop_name} from {target}"
        elif hasattr(restriction, "all_values_from") and restriction.all_values_from:
            target = getattr(restriction.all_values_from, "name", restriction.all_values_from)
            return f"can only have {prop_name} from {target}"
        elif hasattr(restriction, "min_cardinality") and restriction.min_cardinality is not None:
            return f"{prop_name} ≥ {restriction.min_cardinality}"
        elif hasattr(restriction, "max_cardinality") and restriction.max_cardinality is not None:
            return f"{prop_name} ≤ {restriction.max_cardinality}"
        else:
            return f"{prop_name} with unknown restriction {restriction}"

    def expression_to_text(expr):
        if hasattr(expr, "name"):
            return expr.name
        elif "Restriction" in globals() and isinstance(expr, Restriction):
            return restriction_to_text(expr)
        elif "And" in globals() and isinstance(expr, And):
            return " and ".join(expression_to_text(c) for c in expr.Classes)
        elif "Or" in globals() and isinstance(expr, Or):
            return " or ".join(expression_to_text(c) for c in expr.Classes)
        elif "Not" in globals() and isinstance(expr, Not):
            return f"not {expression_to_text(expr.Class)}"
        else:
            return str(expr)

    def class_to_text(cls):
        lines = [f"The class {cls.name} is defined as follows:"]
        for parent in getattr(cls, "is_a", []):
            lines.append(f"- Subclass of {expression_to_text(parent)}")
        for eq in getattr(cls, "equivalent_to", []):
            lines.append(f"- Equivalent to {expression_to_text(eq)}")
        return "\n".join(lines)

    def property_to_text(prop):
        lines = [f"The property {prop.name} is defined as follows:"]
        if hasattr(prop, "domain") and prop.domain:
            domain_text = ", ".join(expression_to_text(d) for d in prop.domain)
            lines.append(f"- Domain: {domain_text}")
        if hasattr(prop, "range") and prop.range:
            range_text = ", ".join(expression_to_text(r) for r in prop.range)
            lines.append(f"- Range: {range_text}")
        if hasattr(prop, "is_functional") and prop.is_functional:
            lines.append("- This property is functional (at most one value).")
        if hasattr(prop, "is_inverse_functional") and prop.is_inverse_functional:
            lines.append("- This property is inverse functional (each value has at most one subject).")
        if hasattr(prop, "is_transitive") and prop.is_transitive:
            lines.append("- This property is transitive.")
        if hasattr(prop, "is_symmetric") and prop.is_symmetric:
            lines.append("- This property is symmetric.")
        if hasattr(prop, "is_asymmetric") and prop.is_asymmetric:
            lines.append("- This property is asymmetric.")
        if hasattr(prop, "is_reflexive") and prop.is_reflexive:
            lines.append("- This property is reflexive (every individual is related to itself).")
        if hasattr(prop, "is_irreflexive") and prop.is_irreflexive:
            lines.append("- This property is irreflexive (no individual is related to itself).")
        return "\n".join(lines)

    def instance_to_text(instance):
        lines = [f"The instance {instance.name} is defined as follows:"]
        types = [cls.name for cls in getattr(instance, "is_a", []) if hasattr(cls, "name")]
        if types:
            lines.append(f"- Belongs to class(es): {', '.join(types)}")
        for prop in instance.get_properties():
            try:
                values = getattr(instance, prop.python_name, [])
            except Exception:
                continue
            if values:
                if not isinstance(values, (list, tuple, set)):
                    values = [values]
                values_text = ", ".join(getattr(v, "name", str(v)) for v in values)
                lines.append(f"- {prop.name}: {values_text}")
        return "\n".join(lines)

    # --- build raw ontology snippets ---
    ontology_texts = []
    for cls in onto.classes():
        ontology_texts.append(class_to_text(cls))
    for prop in onto.object_properties():
        ontology_texts.append(property_to_text(prop))
    for prop in onto.data_properties():
        ontology_texts.append(property_to_text(prop))
    for instance in onto.individuals():
        ontology_texts.append(instance_to_text(instance))

    # --- LLM: convert to human-readable + collect Documents ---
    docs = []
    for text in ontology_texts:
        prompt = (
            "Convert the following ontology snippet into clear, human-readable English.\n"
            f"Snippet: {text}\n\n"
            'Do not add external information or commentary. Only describe what is present. '
            'Do not include any "Here is a snippet" to the output'
        )
        human_text = llm.invoke(prompt)
        out = getattr(human_text, "content", human_text)
        print(out)
        docs.append(Document(page_content=out))

    # --- add to your vector store and return ---
    vector_store.add_documents(docs)
    return vector_store


In [16]:
vector_store = ontology_to_vector_store(onto=onto)

HealthCondition is a subclass of Health.


KeyboardInterrupt: 

In [11]:
def incremental_reasoning(scenarios, vector_store, custom_prompt, llm, k=5):
    """
    Performs step-by-step reasoning on a sequence of scenarios, using accumulated
    story context and ontology-based retrieval for consistency checking and correction.
    """
    results = []
    accumulated_story = ""  # stores all accepted or corrected story so far

    for i, scenario in enumerate(scenarios, start=1):
        # 1. Retrieve ontology documents relevant to the current scenario
        retrieved_docs = vector_store.similarity_search(scenario, k=k)
        docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

        # 2. Construct reasoning context (ontology + accumulated story)
        full_context = (
            f"Ontology context:\n{docs_content}\n\n"
            f"Story so far:\n{accumulated_story.strip() or '(none so far)'}\n"
        )

        # 3. Build the complete prompt using the template
        rendered_prompt = custom_prompt.invoke({
            "question": scenario.strip(),
            "context": full_context.strip(),
        })

        # 4. Invoke the LLM
        answer = llm.invoke(rendered_prompt)
        answer_text = getattr(answer, "content", str(answer)).strip()

        # 5. Parse the model output (Consistency + Updated scenario)
        updated_scenario = scenario.strip()
        consistency_status = "Unknown"

        for line in answer_text.splitlines():
            line_lower = line.strip().lower()
            if line_lower.startswith("updated scenario:"):
                updated_scenario = line.split(":", 1)[1].strip()
            elif line_lower.startswith("consistency:"):
                consistency_status = line.split(":", 1)[1].strip()

        # 6. Ensure updated scenario has fallback if parsing failed
        if not updated_scenario:
            updated_scenario = scenario.strip()

        # 7. Update accumulated story (so the next scenario uses this corrected version)
        accumulated_story = (accumulated_story + "\n" + updated_scenario).strip()

        # 8. Store reasoning results
        results.append({
            "step": i,
            "original_scenario": scenario.strip(),
            "updated_scenario": updated_scenario,
            "consistency": consistency_status,
            "ontology_context": docs_content.strip(),
            "answer_raw": answer_text,
            "story_so_far": accumulated_story,
        })

    return results


In [ ]:
results = incremental_reasoning(scenarios, vector_store, custom_prompt, llm, k=5)
results

[{'step': 1,
  'original_scenario': 'Jack is 235 years old and celebrating his birthday with his family.',
  'updated_scenario': "Jack is 67 years old and celebrating his birthday, but only as an Elder since he's reached that age milestone.",
  'consistency': 'Inconsistent',
  'ontology_context': "Leo belongs to the class of Adults and has the following characteristics:\nHe works as a Waiter at Riccolo.\nHis age is 23 years old.\n\nJohn belongs to the class of Adults and satisfies the following conditions:\n- John is married to Amira.\n- John has a child named Anna.\n- John's age is 34 years old.\n\nThe class Elder is defined as follows:\nElder is a subclass of Thing.\nElder is equivalent to Person and has an age that is at least 67 years old.\n\nMichael belongs to the class of Children and has one parent, Emily.\n\nA person is a thing.\nA person has at least one parent.\nThe age of a person is at most 110 years old.",
  'answer_raw': "Consistency: Inconsistent\nUpdated scenario: Jack 

In [ ]:
prompt = f"Given the story: {story} and previously updated facts of the story: {results} change the original story so that it includes the original but is updated using the new facts"
new_story = llm.invoke(prompt)
new_story

AIMessage(content="Here's the updated story incorporating all the changes:\n\nJack is 67 years old and celebrating his birthday, but only as an Elder since he's reached that age milestone. Despite being considered a reserved person by many, Jack loves talking about himself - it's actually one of his favorite things to do! He's been known to share stories with his family for hours on end.\n\nThis year, Jack's family decided to visit Quito, Ecuador, specifically to see the famous LaVirgenDelPanecillo landmark. The city itself is mostly flat and has large pedestrian zones, making it relatively easy to walk around - although Jack does have some trouble walking far due to his asthma. Luckily, he always carries his medication with him, so whenever he gets out of breath, he can simply take an antihistamine pill to alleviate the symptoms.\n\nAs they stroll through Quito's streets, Jack points out various landmarks and shares stories about his own life experiences. Despite being 67 years old, h

In [19]:
from typing import List, TypedDict, Optional, Dict, Any
from langgraph.graph import StateGraph, START, END
from owlready2 import get_ontology
from langchain_core.prompts import PromptTemplate

class StoryState(TypedDict, total=False):
    story: str
    ontology: Any
    scenarios: List[str]
    results: Any
    new_story: str
    llm: Any
    vector_store: Any

def vector_store_node(state: StoryState) -> Any:
    onto = state.get("ontology")
    vector_store = ontology_to_vector_store(onto)
    return {"vector_store": vector_store}

def split_scenarios_node(state: StoryState) -> Dict[str, Any]:
    story = state.get("story", "")
    scenarios = scenario_splitter_llm(story, llm)
    return {"scenarios": scenarios}

def check_consistency_node(state: StoryState) -> Dict[str, Any]:
    custom_prompt = PromptTemplate.from_template("""
    You are a reasoning assistant that ensures each scenario in a story is logically consistent
    with both (a) the ontology information provided, and (b) the previously verified parts of the story.

    You will be given:
    1. Ontology context — definitions, class restrictions, domains/ranges, and relationships.
    2. The story so far — previously verified or corrected facts.
    3. A new scenario — the next statement to check.

    Your task:
    - Determine whether the new scenario is **consistent** with both the ontology and the story so far.
    - If it is **inconsistent**, rewrite it minimally so that it becomes consistent.
    - Use reasoning — e.g., check domain/range restrictions, disjoint classes, age constraints, or location hierarchies.
    - When fixing inconsistencies, preserve as much of the original story’s meaning as possible.
    - Ensure the corrected version still fits naturally within the ongoing story context.

    Output format (strictly follow this):
    Consistency: [Consistent / Inconsistent]
    Updated scenario: [Rewritten version of the scenario that fits ontology and story context]

    ---

    ### Example 1
    Ontology context:
    - Class Person: a human being.
    - Class Adult ⊆ Person: a person aged 18 or older.
    - Property hasSpouse domain=Adult, range=Adult.

    Story so far:
    John is 15 years old.

    Next scenario to verify:
    John is married to Amira.

    Expected reasoning:
    According to the ontology, only adults can have a spouse. John is 15, so this is inconsistent.

    Expected output:
    Consistency: Inconsistent
    Updated scenario: John is 23 years old and is married to Amira.

    ---

    ### Example 2
    Ontology context:
    - EiffelTower is locatedIn Paris.
    - Paris is locatedIn France.
    - Pisa is locatedIn Italy.

    Story so far:
    John and Amira are on vacation in Italy.

    Next scenario to verify:
    Anna says she wants to visit the Eiffel Tower.

    Expected reasoning:
    The Eiffel Tower is in France, but the story says the family is in Italy.

    Expected output:
    Consistency: Inconsistent
    Updated scenario: Anna says she wants to visit the Tower of Pisa.

    ---

    Now reason about the next scenario using the ontology and story so far.

    {context}

    Next scenario to verify:
    {question}
    """)
    scenarios = state.get("scenarios", [])
    vector_store = state.get("vector_store", [])
    llm = state.get("llm",)
    results = incremental_reasoning(scenarios, vector_store, custom_prompt, llm, k=5)
    return {"results": results}


def fix_story_node(state: StoryState) -> Dict[str, Any]:
    story = state.get("story", "")
    results = state.get("results", [])
    prompt = f"Given the story: {story} and previously updated facts of the story: {results} change the original story so that it includes the original but is updated using the new facts"
    new_story = llm.invoke(prompt)
    return { "new_story": new_story}

builder = StateGraph(StoryState)
builder.add_node("vector_store", vector_store_node)
builder.add_node("split_scenarios", split_scenarios_node)
builder.add_node("check_consistency", check_consistency_node)
builder.add_node("fix_story", fix_story_node)

builder.add_edge(START, "split_scenarios")
builder.add_edge(START, "vector_store")
builder.add_edge("vector_store", "check_consistency")
builder.add_edge("split_scenarios", "check_consistency")
builder.add_edge("check_consistency", "fix_story")
builder.add_edge("fix_story", END)

graph = builder.compile()

onto = get_ontology("./Ontology_Assignment.rdf").load()
story = """John is 15 years old and is on vacation with his wife Amira in Italy. Their daughter Anna can’t wait to visit the Eiffel Tower, but first they will go out to eat. John suggests they eat pizzas since Italy is famous for them, they will eat at restaurant Riccolo located in Florence.
Since Anna has a vitamin C deficiency she will order a pizza that contains lots of vitamin C, so she will get the pizza Bianca because that doesn't contain tomato's so this gives her more vitamin C than a pizza that does contain tomato's. John will get the classic Margherita pizza and Amira orders a pepperoni pizza.
They sit by the window of the small restaurant; the air filled with the smell of garlic and baking dough. Anna swings her legs impatiently under the table, still talking about the Eiffel Tower, while Amira flips through a guidebook about Florence.
15 minutes later the farmer named Leo from the restaurant brings their pizzas and they eat, and Anna says to John how cool it is that the waiter is from France."""

llm = ChatOllama(
    #model="phi3:mini",
    model = "llama3.1"
)

initial_state: StoryState = {
    "story": story,
    "ontology": onto,
    "llm": llm
}
from IPython.display import Image, display
final_state = graph.invoke(initial_state)
print(final_state["new_story"].content)
#display(Image(graph.get_graph(xray=True).draw_mermaid_png()))


The Animal class is a subclass of the class Thing.
The class "Food" is a subclass of "Thing".
The CookingStyles class is a subclass of Thing.
The class Place is a subclass of Thing.
The NoiseLevel class is defined as a subclass of Thing.
The class CharacterTrait is defined as a subclass of Thing.
The class Behaviour is defined as a subclass of Thing.
HealthCondition is a subclass of Health.
The class Nutrient is defined as a subclass of Food.
The class City is defined as follows:

It is a subclass of Place.
A city is located in a country, but it is unknown which specific countries a city is located in.
The Health class is a subclass of Thing.
A condition that is related to food allergies is characterized by:

* It is an allergy
* It comes with symptoms, specifically:
	+ A rash
* It requires a specific medication, namely EPIpen.

Note: This output is based on the provided snippet and assumes it is describing a single condition or concept within the ontology. If there are multiple concep

In [ ]:

from typing import List
from langchain_core.prompts import PromptTemplate

def incremental_reasoning_no_rag(scenarios, custom_prompt, llm):
    """
    Performs step-by-step reasoning on a sequence of scenarios, using only the
    accumulated story so far (no ontology/vector store retrieval).
    """
    results = []
    accumulated_story = ""  # stores all accepted or corrected story so far

    for i, scenario in enumerate(scenarios, start=1):
        # 1. Construct reasoning context (story so far only)
        full_context = f"Story so far:\n{accumulated_story.strip() or '(none so far)'}\n"

        # 2. Build the complete prompt using the template
        rendered_prompt = custom_prompt.invoke({
            "question": scenario.strip(),
            "context": full_context.strip(),
        })

        # 3. Invoke the LLM
        answer = llm.invoke(rendered_prompt)
        answer_text = getattr(answer, "content", str(answer)).strip()

        # 4. Parse the model output (Consistency + Updated scenario)
        updated_scenario = scenario.strip()
        consistency_status = "Unknown"

        for line in answer_text.splitlines():
            line_lower = line.strip().lower()
            if line_lower.startswith("updated scenario:"):
                updated_scenario = line.split(":", 1)[1].strip()
            elif line_lower.startswith("consistency:"):
                consistency_status = line.split(":", 1)[1].strip()

        # 5. Ensure updated scenario has fallback if parsing failed
        if not updated_scenario:
            updated_scenario = scenario.strip()

        # 6. Update accumulated story (so the next scenario uses this corrected version)
        accumulated_story = (accumulated_story + "\n" + updated_scenario).strip()

        # 7. Store reasoning results
        results.append({
            "step": i,
            "original_scenario": scenario.strip(),
            "updated_scenario": updated_scenario,
            "consistency": consistency_status,
            "answer_raw": answer_text,
            "story_so_far": accumulated_story,
        })

    return results



In [ ]:
class StoryStateNoRAG(TypedDict, total=False):
    story: str
    scenarios: List[str]
    results: Any
    new_story_no_rag: Any
    llm: Any

def split_scenarios_node_no_rag(state: StoryStateNoRAG) -> Dict[str, Any]:
    story = state.get("story", "")
    llm = state.get("llm")
    scenarios = scenario_splitter_llm(story, llm)
    return {"scenarios": scenarios}

def check_consistency_node_no_rag(state: StoryStateNoRAG) -> Dict[str, Any]:
    scenarios = state.get("scenarios", [])
    llm = state.get("llm")

    custom_prompt_no_rag = PromptTemplate.from_template("""
    You are a careful reasoning assistant that ensures each scenario in a story
    is logically consistent with the previously verified parts of the same story.

    You will be given:
    1) The story so far — previously verified or corrected facts.
    2) A new scenario — the next statement to check.

    Your task:
    - Determine whether the new scenario is **consistent** with the story so far.
    - If it is **inconsistent**, rewrite it minimally so that it becomes consistent.
    - When fixing inconsistencies, preserve as much of the original meaning as possible.
    - Ensure the corrected version still fits naturally within the ongoing story context.

    Output format (strict):
    Consistency: [Consistent / Inconsistent]
    Updated scenario: [Rewritten version that fits the story context]
    ---

    Story so far:
    {story_so_far}

    Next scenario to verify:
    {next_scenario}
    """)

    results = incremental_reasoning_no_rag(scenarios, custom_prompt_no_rag, llm, k=5)
    return {"results": results}

def fix_story_node_no_rag(state: StoryStateNoRAG) -> Dict[str, Any]:
    story = state.get("story", "")
    results = state.get("results", [])
    llm = state.get("llm")

    updated_facts = "\n".join(f"- {r['updated_scenario']}" for r in results)
    prompt = (
        f"Given the original story:\n{story}\n\n"
        f"And the updated/verified facts:\n{updated_facts}\n\n"
        "Rewrite the story so that it includes the original content but is updated using the new facts. "
        "Keep the tone and style similar to the original and make the result coherent and readable."
    )
    new_story = llm.invoke(prompt)
    return {"new_story_no_rag": new_story}

builder_no_rag = StateGraph(StoryStateNoRAG)
builder_no_rag.add_node("split_scenarios_no_rag", split_scenarios_node_no_rag)
builder_no_rag.add_node("check_consistency_no_rag", check_consistency_node_no_rag)
builder_no_rag.add_node("fix_story_no_rag", fix_story_no_rag)

builder_no_rag.add_edge(START, "split_scenarios_no_rag")
builder_no_rag.add_edge("split_scenarios_no_rag", "check_consistency_no_rag")
builder_no_rag.add_edge("check_consistency_no_rag", "fix_story_no_rag")
builder_no_rag.add_edge("fix_story_no_rag", END)

graph_no_rag = builder_no_rag.compile()

# Example usage (uncomment to run when your LLM & story variables are defined):
# initial_state_no_rag: StoryStateNoRAG = {
#     "story": story,
#     "llm": llm
# }
# final_state_no_rag = graph_no_rag.invoke(initial_state_no_rag)
# print(final_state_no_rag["new_story_no_rag"].content if hasattr(final_state_no_rag["new_story_no_rag"], "content") else final_state_no_rag["new_story_no_rag"])


NameError: name 'TypedDict' is not defined